# 1. Rest Object

In [ ]:
base_url, username, password = ('https://10.10.20.90', 'admin', 'C1sco12345')

In [ ]:
from cisco_sdwan.base.rest_api import Rest

with Rest(base_url, username, password) as api:
    print(api.server_version)

In [ ]:
import json
from cisco_sdwan.base.rest_api import Rest

def print_json(j_obj):
    print(json.dumps(j_obj, indent=2))

with Rest(base_url, username, password) as api:
    reply = api.get('device/control/connections', deviceId="10.10.1.13")
    print_json(reply)

# 2. vManage Models

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(base_url, username, password) as api:
    control_connections = DeviceControlConnections.get(api, deviceId="10.10.1.13")
    print(control_connections)

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(base_url, username, password) as api:
    control_connections = DeviceControlConnections.get(api, deviceId="10.10.1.13")

In [ ]:
control_connections.field_names

In [ ]:
control_connections.field_info('site_id', 'uptime_date')

In [ ]:
help(DeviceControlConnections)

## Adding an extra field to what 'show realtime control connections' would provide

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

with Rest(base_url, username, password) as api:
    control_connections = DeviceControlConnections.get(api, deviceId="10.10.1.13")

    header = control_connections.field_info('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date')
    print(f"{header[0]:15} {header[1]:16} {header[2]:16} {header[3]:5} {header[4]}")
    for row in control_connections.field_value_iter('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date'):
        print(f"{row.system_ip:15} {row.local_color:16} {row.remote_color:16} {row.state:5} {row.uptime_date}")

## Using conversion functions

In [ ]:
from datetime import datetime, timezone
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceControlConnections

# Convert timestamp into date/time string: 2023-06-27 01:46:00 UTC
def datetime_format(timestamp):
    if timestamp is None:
        return ""

    dt = datetime.fromtimestamp(float(timestamp) / 1000, tz=timezone.utc)
    return f"{dt:%Y-%m-%d %H:%M:%S %Z}"

with Rest(base_url, username, password) as api:
    control_connections = DeviceControlConnections.get(api, deviceId="10.10.1.13")

    header = control_connections.field_info('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date')
    print(f"{header[0]:15} {header[1]:16} {header[2]:16} {header[3]:5} {header[4]}")

    for row in control_connections.field_value_iter('system_ip', 'local_color', 'remote_color', 'state', 'uptime_date', uptime_date=datetime_format):
        print(f"{row.system_ip:15} {row.local_color:16} {row.remote_color:16} {row.state:5} {row.uptime_date}")

# 3. Sastre Tasks

In [ ]:
from cisco_sdwan.tasks import implementation

dir(implementation)

## Running 'sdwan show realtime control connections'

In [ ]:
import logging
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.tasks.implementation import TaskShow, ShowRealtimeArgs

# Setup logging to visualize progress
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

# Equivalent to 'sdwan show realtime control connections --system-ip 10.10.1.13'
task_args = ShowRealtimeArgs(
    cmd =  [ 'control', 'connections' ],
    system_ip = "10.10.1.13"
)

with Rest(base_url, username, password) as api:
    task = TaskShow()
    task_output = task.runner(task_args, api)

    if task_output:
        print('\n\n'.join(str(entry) for entry in task_output))

    task.log_info(f'Task completed {task.outcome("successfully", "with caveats: {tally}")}')

## Running 'sdwan backup all'

In [ ]:
import logging
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.tasks.implementation import TaskBackup, BackupArgs

# Setup logging to visualize progress
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

# Equivalent to 'sdwan backup all --no-rollover --save-running'
task_args = BackupArgs(
    save_running = True,
    no_rollover = True,
    workdir = 'backup_test',
    tags = ['all']
)

with Rest(base_url, username, password) as api:
    task = TaskBackup()
    task_output = task.runner(task_args, api)

    if task_output:
        print('\n\n'.join(str(entry) for entry in task_output))

    task.log_info(f'Task completed {task.outcome("successfully", "with caveats: {tally}")}')

## Running 'sdwan restore all'

In [ ]:
import logging
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.tasks.implementation import TaskRestore, RestoreArgs

# Setup logging to visualize progress
logging.basicConfig(level=logging.INFO, format="[%(levelname)s] %(message)s")

# Equivalent to 'sdwan restore all --workdir backup_test'
task_args = RestoreArgs(
    workdir = 'backup_test',
    tag = 'all'
)

with Rest(base_url, username, password) as api:
    task = TaskRestore()
    task_output = task.runner(task_args, api)

    if task_output:
        print('\n\n'.join(str(entry) for entry in task_output))

    task.log_info(f'Task completed {task.outcome("successfully", "with caveats: {tally}")}')

# 4. Implementing functionality not available in Sastre

**Problem Statement:**

Display list of feature templates associated with a particular device template

In [ ]:
from cisco_sdwan.base.rest_api import Rest
from cisco_sdwan.base.models_vmanage import DeviceTemplateIndex, FeatureTemplateIndex, DeviceTemplate, RestAPIException

device_template_name = "Site_3_vEdge_Template"

try:
    with Rest(base_url, username, password) as api:
        device_template_map = {dt_name: dt_id for dt_id, dt_name in DeviceTemplateIndex.get_raise(api)}
    
        device_template_id = device_template_map.get(device_template_name)
        if device_template_id is None:
            raise ValueError(f"Device template '{device_template_name}' was not found")

        device_template = DeviceTemplate.get_raise(api, device_template_id)
        feature_template_map = {ft_id: ft_name for ft_id, ft_name in FeatureTemplateIndex.get_raise(api)}

    print(f"Feature templates used by device template '{device_template_name}':")
    for feature_template_id in device_template.feature_templates:
        print(f"- {feature_template_map.get(feature_template_id, 'not found')}")

except ValueError as ex:
    print(ex)
except RestAPIException as ex:
    print(f"Problem retrieving vManage data: {ex}")